<a href="https://colab.research.google.com/github/heesukjang/W209_DataViz_Summer2023/blob/main/Unit_Assignments/Week_08_Assignment_BubbleChart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## W209 Data Visualization, Summer 2023
**Week 8 Assignment<br>
Heesuk Jang**

### D3, Altair<br>
This weeks assignment is to download your Twitter archive and plot a hashtag network where each node is a hashtag and links are between hashtags that appear in the same tweet.

If you don't tweet, feel free to use the attached FileAttachment("duto_guerra_tweets.json") or a different personal data archive.

### Tableau<br>
Using the same data described above, make a bubble chart with the **size given by the number of edges to each hashtag** and the **color given by the cluster id**. You will need to process this count and the cluster id external to Tableau.

If you don't have a tool in mind, start with Python (e.g., with the networkX library) or TabPy. For GUI only, Gephi is another possibility that should be no problem, but we haven't done this example specifically using Gephi.

In [131]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
import altair as alt
import pandas as pd
import numpy as np
import networkx as nx
import nx_altair as nxa
from itertools import combinations
import json
from collections import defaultdict

# Gdrive Mount using gmail account
from google.colab import drive
drive.mount('/content/gdrive')

pd.options.display.float_format = '{:.0f}'.format
pd.__version__

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'1.5.3'

## Read JSON file from Google Drive (Gmail Account)

In [133]:
with open('/content/gdrive/MyDrive/W209_DataViz/duto_guerra_tweets.json') as f:
  file = f.read()
  data = json.loads(file)

# two hashtags appear together in a tweet
print(f"len(data) = {len(data)}\nhashtags: count_edges = {len(data[181]['entities']['hashtags'])}, {data[181]['entities']['hashtags']}\n")
data[181]

len(data) = 5872
hashtags: count_edges = 3, [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]



{'retweeted': False,
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'entities': {'user_mentions': [{'name': 'RStudio',
    'screen_name': 'rstudio',
    'indices': ['18', '26'],
    'id_str': '235261861',
    'id': '235261861'}],
  'urls': [{'url': 'https://t.co/sN6CjOYIhH',
    'expanded_url': 'http://johnguerra.co/viz/influentials/RStudioConf2018/',
    'display_url': 'johnguerra.co/viz/influentia…',
    'indices': ['114', '137']}],
  'symbols': [],
  'media': [{'expanded_url': 'https://twitter.com/duto_guerra/status/959841361070567425/photo/1',
    'indices': ['160', '183'],
    'url': 'https://t.co/FzHtxIYfVP',
    'media_url': 'http://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'id_str': '959840986485657600',
    'id': '959840986485657600',
    'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'sizes': {'medium': {'w': '600', 'h': '558', 'resize': 'fit'},
     'thumb': {'w': '150', 'h': '150', 're

In [150]:
lt = data[181]['entities']['hashtags']
print("A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):\n\n",lt)

A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):

 [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]


In [151]:
# [{'text': 'dataviz', 'indices': ['109', '117']}]
# [{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
# [{'text': 'dataViz', 'indices': ['119', '127']}]
# [{'text': 'deckgl', 'indices': ['82', '89']}, {'text': 'dataviz', 'indices': ['169', '177']}, {'text': 'WebDev', 'indices': ['178', '185']}]
# [{'text': 'vegaLite', 'indices': ['107', '116']}, {'text': 'WebDev', 'indices': ['136', '143']}]

In [152]:
for i in range(len(data)-5800):
  if len(data[i]['entities']['hashtags']) > 0:
    print(i, data[i]['entities']['hashtags'])

4 [{'text': 'dataviz', 'indices': ['40', '48']}]
17 [{'text': 'dataviz', 'indices': ['225', '233']}]
18 [{'text': 'dataviz', 'indices': ['267', '275']}]
20 [{'text': 'dataviz', 'indices': ['230', '238']}]
21 [{'text': 'dataviz', 'indices': ['96', '104']}]
23 [{'text': 'd3js', 'indices': ['60', '65']}]
26 [{'text': 'CSS', 'indices': ['92', '96']}]
36 [{'text': 'clearviewai', 'indices': ['51', '63']}]
44 [{'text': 'dataviz', 'indices': ['109', '117']}]
47 [{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
50 [{'text': 'dataViz', 'indices': ['119', '127']}]
52 [{'text': 'dataviz', 'indices': ['121', '129']}]
56 [{'text': 'ieeevis', 'indices': ['33', '41']}]
57 [{'text': 'dataviz', 'indices': ['79', '87']}]
58 [{'text': 'eurovis', 'indices': ['48', '56']}, {'text': 'dataviz', 'indices': ['272', '280']}]
63 [{'text': 'eurovis', 'indices': ['31', '39']}]
66 [{'text': 'dataviz', 'indices': ['77', '85']}]
70 [{'text': 'EuroVis2020', 'indices': ['74', '86

## Extract **Nodes** (= Hashtags) and **Edges** between Two Hashtags at a Time

In [140]:
# https://stackoverflow.com/questions/49274565/create-2-tuples-from-a-list
hashtag_nodes = []
hashtag_edges = []

# for i in range(len(data)-5800):
for i in range(len(data)):
  # extract hashtag from each tweet
  hashtag = data[i]['entities']['hashtags']

  # if the hashtag field is not empty, pull the name of hashtag from a tweet, using the key, 'text'
  if len(hashtag) > 0:
    hashtags = [ht['text'].lower() for ht in hashtag]
    # create a set of unique hashtags from tweets
    hashtag_nodes.extend(hashtags)

    # if there are more than one hashtag in a tweet,
    if len(hashtag) > 1:
      hashtag_edge = combinations(hashtags, 2)   # create a list of tuples of two hashtags, representing an edge
      hashtag_edges.extend(sorted(e) for e in hashtag_edge)  # create a master list of sorted edges

print('\nhashtag_nodes: ',set(hashtag_nodes))
print('hashtag_edges: ',hashtag_edges)


hashtag_nodes:  {'webgl', 'd3js', 'oakland', 'sdgatlas', 'openaccess', 'demystifyds', 'dataillustrator', 'cni16f', 'tapestryconf', 'blockbuilder', 'parquesoft', 'uist2020', 'thewebconf', 'vue', 'ionic', 'advice', 'cnn', 'oscarnoms', 'relativism', 'flattenthecurve', 'netviz', 'bogotá', 'eyeo2014', 'pydata', 'ep2014', 'meteor', 'visualanalytics', 'gandissect', 'terps', 'globalwarming', 'msnbc', 'monotrans', 'visweekposterdemo', 'infodesign', 'convocatoria', 'hurricanemichael', 'mckinnovate', 'rstat', 'hashtags', 'serpilopaga', 'bump', 'gii2015', 'internetofthings', 'cityddj', 'keynote', 'vivolatecnología', 'iclr2018', 'iibawards', 'eyeo2019', 'mockusjuegaconcandela', 'python', 'imagine', 'citysight', 'marriageequaility', 'infoviz', 'opensource', 'hug', 'es6', 'photography', 'coldfront18', 'storykit', 'brailleimages', 'lovetwitter', 'lewismumford', 'chart', 'raw', 'ilustrador', 'eco', 'wef', 'bug', 'nasa', 'googlecardboard', 'cscw2019', 'visualizing', 'savefulbright', 'entrepreneurship',

## **Count Edges** Where Two Hashtags Appear Together in a Tweet

In [183]:
edge_dict = defaultdict(int)

for edge in hashtag_edges:
  edge_dict[tuple(edge)] += 1    # count edges
# print(edge_dict)

table =[]
for k, v in edge_dict.items():
  # print(f'k = {k}, v = {v}')
  # count_edge = v
  for node in range(len(k)-1):
    # source_node = k[0]
    # target_node = k[1]
    # print(f'source_node={source_node}, target_node={target_node}\n')
    table_dict = {'source_node':k[0], 'target_node':k[1], 'count_edge':v}
  table.append(table_dict)
table

[{'source_node': 'ar', 'target_node': 'uist2020', 'count_edge': 1},
 {'source_node': 'dataviz', 'target_node': 'eurovis', 'count_edge': 1},
 {'source_node': 'a11y', 'target_node': 'dataviz', 'count_edge': 4},
 {'source_node': 'tactileart',
  'target_node': 'tactiledrawing',
  'count_edge': 1},
 {'source_node': 'nicar18', 'target_node': 'rstats', 'count_edge': 1},
 {'source_node': 'datavis', 'target_node': 'infodesign', 'count_edge': 1},
 {'source_node': 'feedly', 'target_node': 'infovis', 'count_edge': 17},
 {'source_node': 'colorpicker', 'target_node': 'dataviz', 'count_edge': 1},
 {'source_node': 'd3js', 'target_node': 'dataviz', 'count_edge': 32},
 {'source_node': 'dataviz', 'target_node': 'rstudioconf', 'count_edge': 1},
 {'source_node': 'datascience', 'target_node': 'rstudioconf', 'count_edge': 1},
 {'source_node': 'datascience', 'target_node': 'dataviz', 'count_edge': 10},
 {'source_node': 'blind', 'target_node': 'braille', 'count_edge': 1},
 {'source_node': 'atia18', 'target_nod

In [155]:
G = nx.Graph()

# nodes
G.add_nodes_from(hashtag_nodes)
for hashtag in hashtag_nodes:
  # print('hashtag: ',hashtag)
  G.nodes[hashtag]['hashtag'] = hashtag

# edges

for edge in hashtag_edges:
  print('edge: ', edge)
  G.add_edges_from(edge)

edges = []
for edge in G.edges(data=True):
  print(edge)
  # source_node, target_node, weight = edge
  # data_table = {
  #     'source_node'=source_node,
  #     'target_node'=target_node,
  #     'weight'=weight[''],
  # }


# pos = nx.fruchterman_reingold_layout(G)
# viz = nxa.draw_networkx(
#     G=G,
#     pos=pos,
#     node_size=100,
#     # node_tooltip=["hashtag", 'weight'],
#     node_label="hashtag",
#     edge_color="gray",
#     node_color="skyblue",
#     font_size=10

#     #     G=G,
#     # pos=pos,

#     # node_color='steelblue',
#     # edge_color='gray',
#     # node_size=20,
# ).properties(
#     width=700,
#     height=500,
#     title="@duto_guerra's Hashtag Co-occurrence (use mouse to zoom in/out)"
# )

# viz.interactive()

edge:  ['ar', 'uist2020']


NetworkXError: ignored

In [67]:
print(len(l_lt))

2


In [72]:
lt = list()
for inner_list in l_lt:
  # if len(inner_list) == 1:
  lt.append(list(map(list,inner_list)))
print(lt)

[[['d3js', 'dataviz']], [['rstudioconf', 'DataViz'], ['rstudioconf', 'DataScience'], ['DataViz', 'DataScience']]]


### itertools.combinations:
itertools.combinations() provides us with **all the possible tuples** a sequence or set of numbers or letters used in the iterator and the elements are assumed to be unique on the basis of their positions which are distinct for all elements.

In [97]:

# for i in range(len(data)-5800):
#   if len(data[i]['entities']['hashtags']) == 0:
#     continue
#   elif

hashtag_set = set() # For nodes
hashtag_list = list() # For edges

for tweet_dict in data[:200]:
    # print(tweet_dict)
    hashtag_dict = tweet_dict["entities"]["hashtags"]
    # print('hashtag_dict => ', hashtag_dict)
    if len(hashtag_dict) == 0: # No hashtags in this tweet
        continue

    hashtags = [hashtag["text"] for hashtag in hashtag_dict]
    print('hashtags ===> ', hashtags)
    # print('hashtag_set.update(hashtags):  ',hashtag_set.update(hashtags))
    # print(hashtag_set.update(hashtags))
    hashtag_set.update(hashtags)
    # hashtag_set.update(hashtags)
    if len(hashtags) > 1: # Multilpe hashtags in a tweet
        hashtag_list.append(list(itertools.combinations(hashtags, r=2)))
        # print('hashtag_list: ', hashtag_list,'\n')

print('\n========================== RETURN ==========================')
print('NODES, hashtag_set: ',hashtag_set)
print('EDGES, hashtag_list: ',hashtag_list)

# 'hashtags': [{'text': 'WebDev', 'indices': ['16', '23']}, {'text': 'Meteorjs', 'indices': ['96', '105']}, {'text': 'reactjs', 'indices': ['106', '114']}]
# three hashtags: ['WebDev','Meteorjs','reactjs']
# itertool.combinations: [('WebDev', 'Meteorjs'), ('WebDev', 'reactjs'), ('Meteorjs', 'reactjs')]

hashtags ===>  ['dataviz']
hashtags ===>  ['dataviz']
hashtags ===>  ['dataviz']
hashtags ===>  ['dataviz']
hashtags ===>  ['dataviz']
hashtags ===>  ['d3js']
hashtags ===>  ['CSS']
hashtags ===>  ['clearviewai']
hashtags ===>  ['dataviz']
hashtags ===>  ['UIST2020', 'AR']
hashtags ===>  ['dataViz']
hashtags ===>  ['dataviz']
hashtags ===>  ['ieeevis']
hashtags ===>  ['dataviz']
hashtags ===>  ['eurovis', 'dataviz']
hashtags ===>  ['eurovis']
hashtags ===>  ['dataviz']
hashtags ===>  ['EuroVis2020']
hashtags ===>  ['ieeevis']
hashtags ===>  ['BigDataMeetup']
hashtags ===>  ['dataviz', 'a11y']
hashtags ===>  ['proudTeacher']
hashtags ===>  ['TactileArt', 'TactileDrawing']
hashtags ===>  ['NICAR18', 'rstats']
hashtags ===>  ['infodesign', 'datavis']
hashtags ===>  ['ML']
hashtags ===>  ['infovis', 'feedly']
hashtags ===>  ['rstudioconf']
hashtags ===>  ['NoVotoXCorruptos']
hashtags ===>  ['dataviz']
hashtags ===>  ['OpenDataDay']
hashtags ===>  ['NiceXAxis']
hashtags ===>  ['dataviz', 'c

In [30]:
node_ls = []
# for i in range(len(data)-5810):
for i in range(len(data)):
  # print(i, d[i]['entities']['hashtags'])
  for j in range(len(data[i]['entities']['hashtags'])):
    # print('j: ', j)
    if len(data[i]['entities']['hashtags']) > 0:
      # print("data[i]['entities']['hashtags'][j]['text'] => ", data[i]['entities']['hashtags'][j]['text'])
      node_ls.append(data[i]['entities']['hashtags'][j]['text'])

set(node_ls[:25])
node_ls[:20]

['dataviz',
 'dataviz',
 'dataviz',
 'dataviz',
 'dataviz',
 'd3js',
 'CSS',
 'clearviewai',
 'dataviz',
 'UIST2020',
 'AR',
 'dataViz',
 'dataviz',
 'ieeevis',
 'dataviz',
 'eurovis',
 'dataviz',
 'eurovis',
 'dataviz',
 'EuroVis2020']

In [34]:
import collections
count = dict(collections.Counter(node_ls[:20]))
print(f'count.keys() = {count.keys()}\ncount.values() = {count.values()}\n')
count

# defaultdict(int,
#             {'dataviz': 11,
#              'd3js': 1,
#              'css': 1,
#              'clearviewai': 1,
#              'uist2020': 1,
#              'ar': 1,
#              'ieeevis': 1,
#              'eurovis': 2,
#              'eurovis2020': 1})

count.keys() = dict_keys(['dataviz', 'd3js', 'CSS', 'clearviewai', 'UIST2020', 'AR', 'dataViz', 'ieeevis', 'eurovis', 'EuroVis2020'])
count.values() = dict_values([10, 1, 1, 1, 1, 1, 1, 1, 2, 1])



{'dataviz': 10,
 'd3js': 1,
 'CSS': 1,
 'clearviewai': 1,
 'UIST2020': 1,
 'AR': 1,
 'dataViz': 1,
 'ieeevis': 1,
 'eurovis': 2,
 'EuroVis2020': 1}

In [42]:
edge_keys = []
for i in range(len(data)):
    j = 0
    for k in range(1,len(data[i]['entities']['hashtags'])):
      edge_keys.append([data[i]['entities']['hashtags'][j]['text'], data[i]['entities']['hashtags'][k]['text']])
      if k == len(data[i]['entities']['hashtags']) - 1 and k > j+1:
        j += 1
        edge_keys.append(sorted([data[i]['entities']['hashtags'][j]['text'], data[i]['entities']['hashtags'][k]['text']]))

edge_keys[:30]

[['UIST2020', 'AR'],
 ['eurovis', 'dataviz'],
 ['dataviz', 'a11y'],
 ['TactileArt', 'TactileDrawing'],
 ['NICAR18', 'rstats'],
 ['infodesign', 'datavis'],
 ['infovis', 'feedly'],
 ['dataviz', 'colorpicker'],
 ['d3js', 'dataviz'],
 ['rstudioconf', 'DataViz'],
 ['rstudioconf', 'DataScience'],
 ['DataScience', 'DataViz'],
 ['Braille', 'Blind'],
 ['Braille', 'ATIA18'],
 ['ATIA18', 'Blind'],
 ['pokeviz', 'd3js'],
 ['100daysofdataviz', 'dataviz'],
 ['d3js', 'datascience'],
 ['Rusia2018', 'dataviz'],
 ['Rusia2018', 'visualizacion'],
 ['dataviz', 'visualizacion'],
 ['dataviz', 'D3js'],
 ['ieeevis', 'CFP'],
 ['Treemap', 'dataviz'],
 ['Treemap', 'D3js'],
 ['D3js', 'dataviz'],
 ['regl', 'reactjs'],
 ['regl', 'WebDev'],
 ['WebDev', 'reactjs'],
 ['d3js', 'WebDev']]

In [38]:
hashtag_set = set() # For nodes
hashtag_list = list() # For edges

for tweet_dict in data:
    hashtag_dict = tweet_dict["entities"]["hashtags"]
    if len(hashtag_dict) == 0: # No hashtags in this tweet
        continue

    hashtags = [hashtag["text"] for hashtag in hashtag_dict]
    hashtag_set.update(hashtags)
    hashtag_set.update(hashtags)
    if len(hashtags) > 1: # Multilpe hashtags in a tweet
        hashtag_list.append(list(itertools.combinations(hashtags, r=2)))
hashtag_list[:20]

[[('UIST2020', 'AR')],
 [('eurovis', 'dataviz')],
 [('dataviz', 'a11y')],
 [('TactileArt', 'TactileDrawing')],
 [('NICAR18', 'rstats')],
 [('infodesign', 'datavis')],
 [('infovis', 'feedly')],
 [('dataviz', 'colorpicker')],
 [('d3js', 'dataviz')],
 [('rstudioconf', 'DataViz'),
  ('rstudioconf', 'DataScience'),
  ('DataViz', 'DataScience')],
 [('Braille', 'Blind'), ('Braille', 'ATIA18'), ('Blind', 'ATIA18')],
 [('pokeviz', 'd3js')],
 [('100daysofdataviz', 'dataviz')],
 [('d3js', 'datascience')],
 [('Rusia2018', 'dataviz'),
  ('Rusia2018', 'visualizacion'),
  ('dataviz', 'visualizacion')],
 [('dataviz', 'D3js')],
 [('ieeevis', 'CFP')],
 [('Treemap', 'dataviz'), ('Treemap', 'D3js'), ('dataviz', 'D3js')],
 [('regl', 'reactjs'), ('regl', 'WebDev'), ('reactjs', 'WebDev')],
 [('d3js', 'WebDev')]]

In [14]:
from collections import defaultdict
node_dic = defaultdict(int)
for node in node_ls[:20]:
    print('node: ', node)
    node_dic[node] += 1
    print('node_dic[node]: ', node_dic[node], '\n')
node_dic

node:  dataviz
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  2 

node:  dataviz
node_dic[node]:  3 

node:  dataviz
node_dic[node]:  4 

node:  dataviz
node_dic[node]:  5 

node:  d3js
node_dic[node]:  1 

node:  css
node_dic[node]:  1 

node:  clearviewai
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  6 

node:  uist2020
node_dic[node]:  1 

node:  ar
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  7 

node:  dataviz
node_dic[node]:  8 

node:  ieeevis
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  9 

node:  eurovis
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  10 

node:  eurovis
node_dic[node]:  2 

node:  dataviz
node_dic[node]:  11 

node:  eurovis2020
node_dic[node]:  1 



defaultdict(int,
            {'dataviz': 11,
             'd3js': 1,
             'css': 1,
             'clearviewai': 1,
             'uist2020': 1,
             'ar': 1,
             'ieeevis': 1,
             'eurovis': 2,
             'eurovis2020': 1})

In [15]:
[hashtag['text'] for hashtag in data[21]['entities']['hashtags']]

['dataviz']

In [16]:
# question: do we need to consider different letter case hashtags as two different hashtags? i.e. css vs CSS
nodes_ht = []
for i in range(len(data)-5810):
  if len(data[i]['entities']['hashtags']) > 0:
    ht = [hashtag['text'] for hashtag in data[i]['entities']['hashtags']]
    nodes_ht.extend(ht)
set(nodes_ht)

{'AR',
 'CSS',
 'UIST2020',
 'clearviewai',
 'd3js',
 'dataViz',
 'dataviz',
 'eurovis',
 'ieeevis'}